In [13]:
import sys
import os
import itertools
import numpy as np
import pandas as pd

In [43]:
# read dataframe
airports = pd.read_csv('../data/airlines/airports.dat.csv', 
                       names=['Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude', 'Longtitude', 
                              'Altitude', 'Timezone', 'DST', 'Tz database time zone', 'Type', 'Source'])
airlines = pd.read_csv('../data/airlines/airlines.dat.csv',
                       names=['Airline ID', 'Name', 'Alias', 'IATA', 'ICAO', 'Callsign', 'Country', 'Active'])
planes = pd.read_csv('../data/airlines/planes.dat.csv',
                     names=['Name', 'IATA', 'ICAO'])
routes = pd.read_csv('../data/airlines/routes.dat.csv',
                     names=['Airline', 'Airline ID', 'Source airport', 'Source airport ID', 'Destination airport',
                            'Destination airport ID', 'Codeshare', 'Stops', 'Equipment'])
# routes = routes.loc[routes['Source airport ID'] != '\\N']
# routes = routes.loc[routes['Destination airport ID'] != '\\N']
# routes['Source airport ID'] = routes['Source airport ID'].astype('int')
# routes['Destination airport ID'] = routes['Destination airport ID'].astype('int')
airports

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longtitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
5,6,Wewak International Airport,Wewak,Papua New Guinea,WWK,AYWK,-3.583830,143.669006,19,10,U,Pacific/Port_Moresby,airport,OurAirports
6,7,Narsarsuaq Airport,Narssarssuaq,Greenland,UAK,BGBW,61.160500,-45.425999,112,-3,E,America/Godthab,airport,OurAirports
7,8,Godthaab / Nuuk Airport,Godthaab,Greenland,GOH,BGGH,64.190903,-51.678101,283,-3,E,America/Godthab,airport,OurAirports
8,9,Kangerlussuaq Airport,Sondrestrom,Greenland,SFJ,BGSF,67.012222,-50.711603,165,-3,E,America/Godthab,airport,OurAirports
9,10,Thule Air Base,Thule,Greenland,THU,BGTL,76.531197,-68.703201,251,-4,E,America/Thule,airport,OurAirports


In [44]:
routes = routes.merge(airports[['IATA', 'Country']], left_on='Source airport', right_on='IATA')
routes = routes.rename(columns={"Country": "Source country"})
routes = routes.drop(columns=['IATA'])
routes = routes.merge(airports[['IATA', 'Country']], left_on='Destination airport', right_on='IATA')
routes = routes.rename(columns={"Country": "Destination country"})
routes = routes.drop(columns=['IATA'])

In [45]:
countries = pd.unique(pd.concat([routes["Source country"], routes["Destination country"]]))
trans = pd.DataFrame(np.zeros((countries.size, countries.size)), index=countries, columns=countries)
for c1, c2 in routes[["Source country", "Destination country"]].values:
    trans.loc[c1, c2] += 1

In [46]:
with open("../data/countries.txt", 'w') as f:
    for c in countries:
        f.write(c + '\n')


In [79]:
abs_trans = trans.values
total = np.sum(abs_trans, axis=1)
rel_trans = np.zeros(abs_trans.shape)
for i in range(total.size):
    rel_trans[i, :] = abs_trans[i, :] /  total[i]
rel_trans

array([[0.52419793, 0.01903208, 0.04404568, ..., 0.        , 0.        ,
        0.        ],
       [0.05213904, 0.01203209, 0.0026738 , ..., 0.        , 0.        ,
        0.        ],
       [0.58156028, 0.0141844 , 0.14893617, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [81]:
with open('../data/trans.csv', 'w') as f:
    for row in rel_trans:
        f.write(','.join([str(n) for n in row]) + '\n')
